In [ ]:
% load_ext autoreload
% autoreload 2

In [ ]:
import anndata as ad
import numpy as np
from sklearn.linear_model import LinearRegression

from evaluate import evaluate
from preprocessing import recipe_lance22

In [ ]:
dataset_path = "datasets/openproblems_bmmc_cite_phase2_rna/openproblems_bmmc_cite_phase2_rna.censor_dataset.output_"
gex_train = ad.read_h5ad(dataset_path + "train_mod1.h5ad")
gex_test = ad.read_h5ad(dataset_path + "test_mod1.h5ad")
adt_train = ad.read_h5ad(dataset_path + "train_mod2.h5ad")
adt_test = ad.read_h5ad(dataset_path + "test_mod2.h5ad")

In [ ]:
# merge datasets for transductive preprocessing
gex_train_test = ad.concat((gex_train, gex_test), join="outer")

In [ ]:
# apply preprocessing + TSVD dim reduction
gex_train_test = recipe_lance22(gex_train_test, log=False, copy=True)

In [ ]:
# recover train test split from processed data
X_train = gex_train_test.obsm["X_pca"][:len(gex_train)]
X_test = gex_train_test.obsm["X_pca"][len(gex_train):]

In [ ]:
# adt targets
Y_train = adt_train.X.toarray()
Y_test = adt_test.X.toarray()

In [ ]:
# train linear regression model
model = LinearRegression()
model.fit(X_train, Y_train)

In [ ]:
# predict on the test set
Y_pred = model.predict(X_test)
Y_pred = np.clip(Y_pred, a_min=0, a_max=None)

In [ ]:
evaluate(Y_pred, Y_test)